In [1]:
import os
import shutil
#import torch
import cv2

Load videos from Cord Platform

In [2]:
from encord.client import EncordClient
client = EncordClient.initialise(
  'ff31ec11-fd68-4db9-a8c2-a4e97cbb4b7e',  # Project ID
  'cR2DFqKUVsnZCzJ_xvG7ZkxniWv7MeINnAYI8dNMnHQ'  # API key
)
# Get and print project info (labels, datasets)
project = client.get_project()
print(project)

{'project_hash': 'ff31ec11-fd68-4db9-a8c2-a4e97cbb4b7e', 'title': 'SurgEase - OGD project ', 'description': '', 'editor_ontology': {'objects': [{'id': '1', 'name': 'Epiglottis ', 'color': '#D33115', 'shape': 'bounding_box', 'featureNodeHash': '249c9370'}, {'id': '2', 'name': 'Larynx', 'color': '#E27300', 'shape': 'bounding_box', 'featureNodeHash': 'e3c87d43'}, {'id': '3', 'name': 'Oesophagus ', 'color': '#16406C', 'shape': 'bounding_box', 'featureNodeHash': 'c6eb717a'}, {'id': '4', 'name': 'Z-line ', 'color': '#FE9200', 'shape': 'polyline', 'featureNodeHash': '8b64f336'}, {'id': '5', 'name': 'Stomach', 'color': '#FCDC00', 'shape': 'bounding_box', 'featureNodeHash': 'f44083a3'}, {'id': '6', 'name': 'Incisura ', 'color': '#A4DD00', 'shape': 'polygon', 'featureNodeHash': '8dd583bb'}, {'id': '7', 'name': 'D1', 'color': '#68CCCA', 'shape': 'bounding_box', 'featureNodeHash': '58bcccb6'}, {'id': '8', 'name': 'D2', 'color': '#73D8FF', 'shape': 'bounding_box', 'featureNodeHash': '761eff9d'}, {'

Get labeled video names

In [12]:
video_name_list= []
all_video_name_list = [item["data_title"] for item in project["label_rows"]]
#print(all_video_name_list)
print('Total number of video is: ',len(all_video_name_list))
#labeled_video_name_list = sorted([video_info["data_title"] for video_info in project["label_rows"] if video_info["label_status"] == "LABELLED"])
completed_video_name_list = sorted([video_info["data_title"] for video_info in project["label_rows"] if video_info["annotation_task_status"] == "COMPLETED"])
print('Number of completed video is: ',len(completed_video_name_list))
reviewed_video_name_list = sorted([video_info["data_title"] for video_info in project["label_rows"] if video_info["annotation_task_status"] == "IN_REVIEW"])
print('Number of reviewed video is: ',len(reviewed_video_name_list))

ready_to_use_video_name_list = ['2015-07-044.mp4', '2015-07-046.mp4', '2015-07-079.mp4', '2015-08-0106.mp4', '2015-08-034.mp4', '2015-08-050.mp4', 
                                '2015-08-056.mp4', '2015-09-0108.mp4', '2015-09-066.mp4', '2015-09-080.mp4', '2015-09-084.mp4', '2015-09-095.mp4', 
                                '2015-11-030.mp4', '2015-11-055.mp4', '2015-11-068.mp4', '2015-11-09.mp4', '2015-12-62.mp4', '2016-07-042.mp4', 
                                '2016-08-0117.mp4', '1_ENDOSCOPY_01 (video-converter.com).mp4', '2015-06-089.mp4', '2015-06-0101.mp4', '015-06-0122.mp4',
                                '2015-07-044.mp4', ' 2015-07-046.mp4', '2015-08-016.mp4', '2015-08-013.mp4', '2015-08-056.mp4', '2015-08-095.mp4',
                                '2015-09-01.mp4', '2015-09-084.mp4', '2015-09-0108.mp4', '2015-10-060.mp4', '2015-11-030.mp4', '2015-11-068.mp4', 
                                '2015-12-62.mp4','2016-01-029.mp4', '2016-01-089.mp4', '2016-02-043.mp4', '2016-02-049.mp4', '2016-02-068.mp4', 
                                '2016-03-047.mp4', '2016-03-066.mp4', '2016-04-038.mp4', '2016-05-07.mp4', '2016-05-0125.mp4', '2016-06-023.mp4',
                                '2016-06-060.mp4', '2016-08-040.mp4', '2016-09-033.mp4', '2016-09-052.mp4', '2016-09-0142.mp4', '2016-10-051.mp4',
                                '2016-10-060.mp4', '2016-10-066.mp4', '2016-11-033.mp4', '2017-01-069.mp4', '2017-03-076.mp4', '2017-04-012.mp4',
                                '2017-05-0102.mp4', '2017-05-0113.mp4']
print('Number of ready to use(reviewed&completed) video is: ',len(ready_to_use_video_name_list))

Total number of video is:  466
Number of completed video is:  19
Number of reviewed video is:  406
Number of ready to use(reviewed&completed) video is:  61


In [13]:
video_url_list = []
delete_index = []
for i, video in enumerate(ready_to_use_video_name_list):
    item=0
    for no in range(len(project['label_rows'])):
        if project['label_rows'][no]["data_title"] == video:
            item = no
    item_label_hash = project['label_rows'][item]['label_hash']
    item_data_hash = project['label_rows'][item]['data_hash']
    try:
        data_row = client.get_data(item_data_hash, get_signed_url=True)
        signed_url = data_row[0]['file_link']
        video_url_list.append(signed_url)
    except:
        delete_index.append(i)

for index in delete_index:
    del ready_to_use_video_name_list[index]

print('Number of videos can not read is:', len(delete_index))
print('Number of used labeled video is:', len(ready_to_use_video_name_list))
print('Length of url_list is:', len(video_url_list))

Number of videos can not read is: 0
Number of used labeled video is: 61
Length of url_list is: 61


In [15]:
#Save video names and signed urls
import pickle
with open("labeled_video_name_list", "wb") as fp:   #Pickling
    pickle.dump(ready_to_use_video_name_list, fp)
with open("video_url_list", "wb") as fp:   #Pickling
    pickle.dump(video_url_list, fp)

In [16]:
#Load video names and signed urls && Change environment cv2
with open("labeled_video_name_list", "rb") as fp:   # Unpickling
    ready_to_use_video_name_list = pickle.load(fp)
with open("video_url_list", "rb") as fp:   # Unpickling
    video_url_list = pickle.load(fp)
print(ready_to_use_video_name_list)

['2015-07-044.mp4', '2015-07-046.mp4', '2015-07-079.mp4', '2015-08-0106.mp4', '2015-08-034.mp4', '2015-08-050.mp4', '2015-08-056.mp4', '2015-09-0108.mp4', '2015-09-066.mp4', '2015-09-080.mp4', '2015-09-084.mp4', '2015-09-095.mp4', '2015-11-030.mp4', '2015-11-055.mp4', '2015-11-068.mp4', '2015-11-09.mp4', '2015-12-62.mp4', '2016-07-042.mp4', '2016-08-0117.mp4', '1_ENDOSCOPY_01 (video-converter.com).mp4', '2015-06-089.mp4', '2015-06-0101.mp4', '015-06-0122.mp4', '2015-07-044.mp4', ' 2015-07-046.mp4', '2015-08-016.mp4', '2015-08-013.mp4', '2015-08-056.mp4', '2015-08-095.mp4', '2015-09-01.mp4', '2015-09-084.mp4', '2015-09-0108.mp4', '2015-10-060.mp4', '2015-11-030.mp4', '2015-11-068.mp4', '2015-12-62.mp4', '2016-01-029.mp4', '2016-01-089.mp4', '2016-02-043.mp4', '2016-02-049.mp4', '2016-02-068.mp4', '2016-03-047.mp4', '2016-03-066.mp4', '2016-04-038.mp4', '2016-05-07.mp4', '2016-05-0125.mp4', '2016-06-023.mp4', '2016-06-060.mp4', '2016-08-040.mp4', '2016-09-033.mp4', '2016-09-052.mp4', '20

In [20]:
def create_frame_from_videos(target_folder, video_url_list, video_name_list):
    for video_url,video_name in zip(video_url_list, video_name_list):

        frames_path = os.path.join(target_folder, video_name, "frames")
        
        if os.path.exists(frames_path):
            shutil.rmtree(frames_path)
        os.makedirs(frames_path)
        print(video_name)

        video = cv2.VideoCapture(video_url)

        frame_count = 0
        while(video.isOpened()):
            ret, frame = video.read()
            if(not ret):    #There is one video can not read
                break
            #frame_resized = cv2.resize(frame, (960, 540))
            cv2.imwrite(os.path.join(frames_path, str(frame_count)+".png"), frame)
            frame_count += 1
        video.release()

target_folder=r'..\Dataset\data_all_frames_540x960'
create_frame_from_videos(target_folder, video_url_list, ready_to_use_video_name_list)

2015-07-044.mp4
2015-07-046.mp4
2015-07-079.mp4
2015-08-0106.mp4
2015-08-034.mp4
2015-08-050.mp4
2015-08-056.mp4
2015-09-0108.mp4
2015-09-066.mp4
2015-09-080.mp4
2015-09-084.mp4
2015-09-095.mp4
2015-11-030.mp4
2015-11-055.mp4
2015-11-068.mp4
2015-11-09.mp4
2015-12-62.mp4
2016-07-042.mp4
2016-08-0117.mp4
1_ENDOSCOPY_01 (video-converter.com).mp4
2015-06-089.mp4
2015-06-0101.mp4
015-06-0122.mp4
2015-07-044.mp4
 2015-07-046.mp4
2015-08-016.mp4
2015-08-013.mp4
2015-08-056.mp4
2015-08-095.mp4
2015-09-01.mp4
2015-09-084.mp4
2015-09-0108.mp4
2015-10-060.mp4
2015-11-030.mp4
2015-11-068.mp4
2015-12-62.mp4
2016-01-029.mp4
2016-01-089.mp4
2016-02-043.mp4
2016-02-049.mp4
2016-02-068.mp4
2016-03-047.mp4
2016-03-066.mp4
2016-04-038.mp4
2016-05-07.mp4
2016-05-0125.mp4
2016-06-023.mp4
2016-06-060.mp4
2016-08-040.mp4
2016-09-033.mp4
2016-09-052.mp4
2016-09-0142.mp4
2016-10-051.mp4
2016-10-060.mp4
2016-10-066.mp4
2016-11-033.mp4
2017-01-069.mp4
2017-03-076.mp4
2017-04-012.mp4
2017-05-0102.mp4
2017-05-011